In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip install torchaudio
    !pip install wandb --upgrade
    # !wandb login
    # !pip install umap-learn
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/temporal')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Extra_and_Unused')
    root = "/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project"
else:
    print("Running local")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "media/mirp_ai/Seagate Desktop Drive/Jaguas_2018"

Running local


In [2]:
# from __future__ import print_function
# from __future__ import print_function
import os
import numpy as np

import datetime
import gc

from scipy import signal

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torchaudio.transforms as audio_transform

from Jaguas_DataLoader_rainless import SoundscapeData
from Models import ConvAE as AE
# from AE_training_functions import TestModel, TrainModel # For AE
# from PosAE_training_functions import posautoencoding_m1 as AE, TestModel, TrainModel # For PosAE

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = xm.xla_device()
print(device)

from datetime import timedelta
import joypy
from sklearn.preprocessing import StandardScaler
import pandas as pd
import umap
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# wandb.login()

cuda


In [3]:
# Batch methods
from sklearn.cluster import KMeans
from sklearn.cluster import Birch
import matplotlib.cm as cm

# Single methods
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph
from sklearn import metrics
from sklearn.metrics import silhouette_samples

In [4]:
def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = os.listdir(path_to_dir)
    return sorted([ filename for filename in filenames if filename.endswith( suffix ) ])

In [5]:
AI_path = f"{root}/Jaguas/Complementary_Files/Acoustic_Indices"
files = find_csv_filenames(AI_path, suffix=".csv")
files = files[1::]

In [12]:
AI = pd.read_csv(f"{AI_path}/{files[0]}", index_col=0)
total = len(AI)
for file in files[1::]:
    df_file = pd.read_csv(f"{AI_path}/{file}")
    total += len(df_file)
    print(file)
    AI = AI.append(df_file)
for i in range(len(AI)):
    AI.iloc[i,0] = AI.iloc[i,0].split("/")[-1]
AI.reset_index(inplace=True)
AI.drop(columns="index", inplace=True)
AI.to_csv(f"{root}/Jaguas/Complementary_Files/Acoustic_Indices/AI_Jaguas.csv")

G04_m_AIs.csv
G06_m_AIs.csv
G07_m_AIs.csv
G08_m_AIs.csv
G09_m_AIs.csv
G13_m_AIs.csv
G15_m_AIs.csv
G17_m_AIs.csv
G19_m_AIs.csv
G23_m_AIs.csv
G24_m_AIs.csv
G25_m_AIs.csv
G27_m_AIs.csv
G28_m_AIs.csv
G29_m_AIs.csv
G34_m_AIs.csv
G35_m_AIs.csv


/tmp/ipykernel_775009/194938973.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  AI = AI.append(df_file)
/tmp/ipykernel_775009/194938973.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  AI = AI.append(df_file)
/tmp/ipykernel_775009/194938973.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  AI = AI.append(df_file)
/tmp/ipykernel_775009/194938973.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  AI = AI.append(df_file)
/tmp/ipykernel_775009/194938973.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  AI = AI.append(df_file)
/tmp/ipykernel_

G36_m_AIs.csv
G37_m_AIs.csv
G40_m_AIs.csv
G41_m_AIs.csv
G46_m_AIs.csv
G47_m_AIs.csv
G49_m_AIs.csv
G50_m_AIs.csv
G51_m_AIs.csv
G52_m_AIs.csv
G54_m_AIs.csv
G57_m_AIs.csv
G58_m_AIs.csv


/tmp/ipykernel_775009/194938973.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  AI = AI.append(df_file)
/tmp/ipykernel_775009/194938973.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  AI = AI.append(df_file)
/tmp/ipykernel_775009/194938973.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  AI = AI.append(df_file)
/tmp/ipykernel_775009/194938973.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  AI = AI.append(df_file)
/tmp/ipykernel_775009/194938973.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  AI = AI.append(df_file)
/tmp/ipykernel_

In [ ]:
base_path = f"{root}/Jaguas/Complementary_Files/Audios_Jaguas/Audios_Jaguas.csv"
base = pd.read_csv(base_path, index_col=0)

In [ ]:
base

In [ ]:
corrupted = []
for i in range(len(base)):
    file = base.iloc[i,4]
    columns = np.arange(2,62)
    try:
        base[AI.columns[columns]] = AI[AI["file"] == file].iloc[0,columns]
    except:
        corrupted.append(i)
        print(file)
base = base.drop(corrupted)
# base.to_csv(f"{root}/Jaguas/Complementary_Files/Acoustic_Indices/AI_Jaguas.csv")

In [ ]:
base2 = base.drop(columns=["Time", "Recording", "Habitat",
                           "Recorder", "Date", "Municipio",
                           "Vereda", "Latitud", "Rain_occurrence",
                           "Rain_Intesity", "Intensity_Category",
                           "Longitud", "Elevacion"])

In [ ]:
len(base2)

In [ ]:
AI_array = base2.drop(columns="Filename").to_numpy()

In [ ]:
AI_array.shape

In [ ]:
list(base["Time"])

In [ ]:
y_recorder = list(base["Recorder"])
y_hour = list(base["Time"])
for i in range(len(y_hour)):
    y_hour[i] = int(y_hour[i].split(":")[0])
    y_recorder[i] = int(y_recorder[i][1:3])
print(len(y_hour), len(y_recorder))
y = {"hour": y_hour, "recorder": y_recorder}

In [ ]:
from logging import raiseExceptions
import librosa
def plot_silhouette( X, cluster_labels, n_clusters, silhouette_avg, method, extra="", save=False):
    fig, ax1 = plt.subplots(figsize=(12, 12))

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-1, -0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1, 0,
                    0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1])
    if save == True:
        plt.savefig(f"temporal/clustering_results/{method}/Silhouette_plot_{n_clusters}.pdf", format="pdf")
        plt.show()
    else:
        print("Ploted!")
        pass

def plot_centroids(cluster_centers, model, method, extra="", save=True):
    plt.figure(figsize=(18, 18))
    testing._model.to("cpu")
    for i, spec in enumerate(cluster_centers):
        encodings = spec.reshape(64,9,9)
        encodings = torch.tensor(encodings).float()
        decodings = testing._model.decoder(encodings).detach().numpy()
        plt.subplot(6, 6, i + 1)
        plt.imshow(librosa.power_to_db(decodings[0, :, :]), origin="lower", cmap="viridis")
        plt.xticks(())
        plt.yticks(())
    n_cluster = len(cluster_centers)
    if save == True:
        plt.savefig(f"temporal/clustering_results/{method}/Centroids_plot_{n_cluster}_{extra}.pdf", format="pdf")
        plt.show()
    else:
        print("Ploted!")
        pass
    

import math

def num_rows_cols(num_elements):
    num_rows = int(math.sqrt(num_elements))
    num_cols = (num_elements + num_rows - 1) // num_rows
    return (num_rows, num_cols)

def get_row_col(pos, cols):
    row = pos // cols
    col = pos % cols
    return row, col

class Clustering_Results:
    def __init__(self, model, y, y_label="hour", hist_library="plt"):
        self._labels_cluster = None
        self._hist_library = hist_library
        self._label = y_label
        self._model = model
        self._n_clusters = len(set(model.labels_))
        self.y = y
        self._y = np.asarray(y[self._label])
        self._n_labels = set(self._y)


    def one_cluster_eval(self, cluster):
        index = np.where(self._model.labels_ == cluster)
        index = list(index[0])
        self._labels_cluster = self._y[index]
        return self._labels_cluster
    
    def tagger(self, samples):
        labels = []
        labels_all_clusters = []
        joy_vars = ["hour", "recorder"]
        for cluster in range(self._n_clusters):
            y_aux = []
            labels_cluster = []
            for i, label in enumerate(joy_vars):
                y_aux.append(self.converter(self.y[label]))
                index = np.where(self._model.labels_ == cluster)
                index = list(index[0])
            labels.append(samples[index])
        return labels

    def joyplot(self):
        labels_all_clusters = []
        size_x = 8
        size_y = 6
        joy_vars = ["hour", "recorder"]
        for cluster in range(self._n_clusters):
            y_aux = []
            labels_cluster = []
            for i, label in enumerate(joy_vars):
                y_aux.append(self.converter(self.y[label]))
                index = np.where(self._model.labels_ == cluster)
                index = list(index[0])
                labels_cluster.append(y_aux[i][index])
            df = pd.DataFrame({'recorder':labels_cluster[0], "hour":labels_cluster[1]})
            joypy.joyplot(df, by="hour", column="recorder", range_style='own', 
                            grid="y", hist=False, linewidth=1, legend=False, figsize=(size_x,size_y),
                            title=f"Cluster {cluster} \nLabels distribution along recorders using recorders as rows",
                            colormap=cm.autumn_r, fade=False)
            joypy.joyplot(df, by="recorder", column="hour", range_style='own', 
                                grid="y", hist=False, linewidth=1, legend=False, figsize=(size_x,size_y),
                                title=f"Cluster {cluster} \nLabels distribution along recorders using hours as rows",
                                colormap=cm.autumn_r)
            labels_all_clusters.append(index)
            plt.show()
#             print(len(labels_cluster))
#             print(labels_cluster[1].shape)
#             print(labels_cluster[0:10])
#             print(index[0:20])
        
        return labels_all_clusters
            

    def histograms(self):
        bins = list(self._n_labels)
        num_rows, num_cols = num_rows_cols(self._n_clusters)
        fig, axes = plt.subplots(num_rows, num_cols, figsize=(14, 14))
        if self._n_clusters <= 3:
                axes = np.expand_dims(axes,0)
                fig.set_figheight(6)
                fig.set_figwidth(12)
                if self._n_clusters == 1:
                    axes = np.expand_dims(axes,0)
                else:
                    pass
        else:
            pass
        for hist in range(self._n_clusters):
            aux = self.one_cluster_eval(hist)
            ax_0, ax_1 = get_row_col(hist, num_cols)
            if self._hist_library == "plt":
                axes[ax_0][ax_1].hist(aux, histtype="bar",
                                      color="paleturquoise", cumulative=False,
                                      edgecolor='black', 
                                      linewidth=1.2, bins=bins, stacked=False)
                axes[ax_0][ax_1].set_title(f"Cluster: {hist}", size=16)
            elif self._hist_library == "sns":
                sns.distplot(aux,bins=np.arange(aux.min(), aux.max()+1),
                             hist_kws=dict(edgecolor="black", linewidth=1), 
                             ax=axes[ax_0, ax_1])
                axes[ax_0][ax_1].set_title(f"Cluster: {hist}", size=16)              
            else:
                raise Exception(f"Library {self._hist_library} unused")
        plt.show()

In [ ]:
X = AI_array
scaler = StandardScaler().fit(X)
X_n = X
X_scaled = scaler.transform(X)
# X_PCA = PCA(n_components=180).fit_transform(X_scaled)
X_TSNE = TSNE(n_components=2, learning_rate="auto", init='random', random_state=0).fit_transform(X_scaled)

In [ ]:
n_components = 60
X_PCA_ = PCA(n_components=n_components).fit(X)
X_PCA = X_PCA_.transform(X)
plt.figure(figsize=(10,8))
plt.plot(range(0, n_components), X_PCA_.explained_variance_ratio_.cumsum())
plt.title("Explained variance by components")
plt.xlabel("Number of components")
plt.ylabel("Cummulative Explained Variance ")
plt.axhline(y=0.9, c="r", linestyle="--")

In [ ]:
Xr = np.random.rand(2068, 60)
Xr.shape

In [ ]:
from sklearn.cluster import KMeans
import seaborn as sns
Kmeans = KMeans(n_clusters=10, random_state=0).fit(X_TSNE)
import warnings
warnings.filterwarnings("ignore")
Kmeans_Results = Clustering_Results(Kmeans, y, y_label="recorder", hist_library="plt")

In [ ]:
Kmeans_Results.histograms()